# Greatness Rating for NBA players

### Observations:
1. Great players tend to lead the league annually in 'primary' categories.
2. Great players tend to lead the league annually in 'secondary' categories.  However, these categories are subordinate to 'primary' categories for the reasons below, and should not carry the same weight as 'primary' categories.
    1. Percentage based 'secondary' categories are susceptible to inclusion of specialty situation players who are non-starters, e.g. a low volume shooter with a high field goal percentage.
    2. Other 'secondary' categories are mere subcategories or combinations of 'primary' categories, e.g. three-point field goals is a subcategory of scoring, and triple-doubles is a combination of 3 primary categories.
3. Great players are made greater by winning team championships.
4. It's more difficult to be a great player when there is more competition.  i.e. BIG fish in a small pond.


### Rules:
1. Player is awarded a point each time he leads league annually in one of the 'primary' categories: 
    1. scoring
    2. rebounding
    3. assists
    4. steals
    5. blocks
2. Player is awarded 1/2 point each time he leads league annually in one of the 'secondary' categories:
    1. triple-doubles
    2. field goal percentage
    3. free throw percentage
    4. three-point field goals
    5. three-point field goal percentage
3. Player is awarded a point for each team championship, not to exceed the number of points already awarded.
4. Player's points are normalized based on the number of teams in the league during his playing career.  
   
Player's points are calculated as below:  
   _subtotal = primary + 0.5 * seondary_  
   _total = subtotal + min(championships, subtotal)_  
   _normalized = total * median_player_teams / max_teams_

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# -*- encoding: utf-8 -*-
%matplotlib inline

Imports

In [3]:
from datetime import datetime
import numpy as np
import pandas as pd
import os
import re
import shutil
import wikipedia as wp

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_rows', 500)

In [4]:
# Make sure your code works only with Unicode strings internally,
# converting to a particular encoding on output,
# and decoding str on input. Learn the libraries you are using,
# and find places where they return str. Decode str before return
#value is passed further in your code.
#https://gist.github.com/gornostal/1f123aaf838506038710
def force_to_unicode(text):
    "If text is unicode, it is returned as is. If it's str, convert it to Unicode using UTF-8 encoding"
    return text if isinstance(text, unicode) else text.decode('utf8')

def _fix_name_variations(name):
    
    #Kareem Abdul-Jabbar, Lew Alcindor
    #Magic Johnson, Earvin "Magic" Johnson
    if name in (u'Lew Alcindor'):
        name = u'Kareem Abdul-Jabbar'
    elif name in (u'Magic Johnson'):
        name = u'Earvin "Magic" Johnson'
    elif name in (u'Don "Slick" Watts'):
        name = u'Slick Watts'
    return force_to_unicode(name)

def _fix_name(player):
    name = u''
    #Abdul-Jabbar, KareemKareem Abdul-Jabbar* (2)
    #Archibald, Nate "Tiny"Nate "Tiny" Archibald*
    #O'Neal, ShaquilleShaquille O'Neal*
    #Jones, K. C.K. C. Jones*[b]
    
    # This looks for a sequence of 1 or more arbitrary characters (.+?)
    # (as a non-greedy match, so that it tries shorter sequences first),
    # followed by 1 or more repetitions of the matched sequence \1+,
    # and replaces it all with just the matched sequence \1.
    name = re.sub(r'(.{2,}?)\1+', r'\1', force_to_unicode(player))
    
    # Abdul-Jabar, Kareem Abdul-Jabar* (2)
    try:
        name = re.split(',\s+', name)[1]
    except IndexError:
        pass
    
    # Kareem Abdul-Jabar*
    m = re.match(r'([^*^\^^\[^\(]+)', name)
    if m:
        name = m.group(1)
    
    # remove trailing whitespace
    name = name.strip()
    
    # special cases of more than one name for a person
    name = _fix_name_variations(name)

    return force_to_unicode(name)

def _fix_table(filename):

    out = open(filename + '~', 'w')
    
    with open(filename, 'r') as f:
        for line in f:
            if line:
                line = line.decode('utf-8')
                rank = re.split(',', line)[0]
                if rank.isdigit():
                    old_rank = rank
                elif u'Rank' not in rank:
                    line = u','.join([old_rank, line])
                    line=line[::-1].replace(",","",1)[::-1]

                out.write(line.encode('utf-8'))
    out.close()
    shutil.move(filename + '~', filename)

def get_table(title, filename, match):

    if os.path.isfile(filename):
        pass
    else:
        html = wp.page(title).html()
        df = pd.read_html(html, header=0, match=match)[0]
        
        if filename not in ('championships.csv'):
            df['Player'] = df['Player'].apply(_fix_name)
        
        df.to_csv(filename, header=True, index=False, encoding='utf-8')
        
        if filename in ('championships.csv'):
            _fix_table(filename)
            df = pd.read_csv(filename)
                
            df['Player'] = df['Player'].apply(_fix_name)
            df.to_csv(filename, header=True, index=False, encoding='utf-8')
            
    df = pd.read_csv(filename)
    return df

In [5]:
title = 'List of NBA players with most championships'
filename = 'championships.csv'
championships = get_table(title, filename, match='Championship teams')
print(title)
championships.head()

List of NBA players with most championships


,Rank,Player,Pos,Seasons played,Total won,Championship teams,Ref.
0,1,Bill Russell,C,13,11,"Boston Celtics (1957, 1959, 1960, 1961, 1962, ...",[26]
1,2,Sam Jones,G,12,10,"Boston Celtics (1959, 1960, 1961, 1962, 1963, ...",[9]
2,3,Tom Heinsohn,F/C,9,8,"Boston Celtics (1957, 1959, 1960, 1961, 1962, ...",[10]
3,3,K. C. Jones,G,9,8,"Boston Celtics (1959, 1960, 1961, 1962, 1963, ...",[11]
4,3,"Tom ""Satch"" Sanders",F,13,8,"Boston Celtics (1961, 1962, 1963, 1964, 1965, ...",[12]


In [6]:
title = 'List of National Basketball Association annual scoring leaders'
filename = 'scoring.csv'
scoring = get_table(title, filename, match='Joe Fulks')
print(title)
scoring.head()

List of National Basketball Association annual scoring leaders


,Season,Player,Age[a],Pos[b],Team,Gamesplayed,Field goalsmade,3-pointfield goalsmade,Free throwsmade,Totalpoints,Points pergame,Ref
0,1946–47,Joe Fulks,25,F/C,Philadelphia Warriors,60,475,—,439,1389,23.2,[11][12]
1,1947–48,Max Zaslofsky,22,G/F,Chicago Stags,48,373,—,261,1007,21.0,[14][15]
2,1948–49,George Mikan,24,C,Minneapolis Lakers,60,583,—,532,1698,28.3,[16][17]
3,1949–50,George Mikan,25,C,Minneapolis Lakers,68,649,—,567,1865,27.4,[17][18]
4,1950–51,George Mikan,26,C,Minneapolis Lakers,68,678,—,576,1932,28.4,[17][19]


In [7]:
title = 'List of National Basketball Association annual rebounding leaders'
filename = 'rebounding.csv'
rebounding = get_table(title, filename, match='Dolph Schayes')
print(title)
rebounding.head()

List of National Basketball Association annual rebounding leaders


,Season,Player,Pos.[a],Team,Gamesplayed,Offensiverebounds,Defensiverebounds,Totalrebounds,Reboundsper game,References
0,1950–51,Dolph Schayes,F/C,Syracuse Nationals,66,—,—,1080,16.36,[8][9]
1,1951–52,Larry Foust,C,Fort Wayne Pistons,66,—,—,880,13.33,[8][10]
2,1951–52,Mel Hutchins,C,Milwaukee Hawks,66,—,—,880,13.33,[8][11]
3,1952–53,George Mikan,C,Minneapolis Lakers,70,—,—,1007,14.39,[8][12]
4,1953–54,Harry Gallatin,F/C,New York Knicks,72,—,—,1098,15.25,[8][13]


In [8]:
title = 'List of National Basketball Association annual assists leaders'
filename = 'assists.csv'
assists = get_table(title, filename, match='Ernie Calverley')
print(title)
assists.head()

List of National Basketball Association annual assists leaders


,Season,Player,Position[a],Team,Gamesplayed,Totalassists,Assistsper game,References
0,1946–47,Ernie Calverley,G,Providence Steamrollers,59,202,3.42,[5][6]
1,1947–48,Howie Dallmar,F,Philadelphia Warriors,48,120,2.50,[5][7]
2,1948–49,Bob Davies,G/F,Rochester Royals,60,321,5.35,[5][8]
3,1949–50,Dick McGuire,G,New York Knicks,68,386,5.68,[5][9]
4,1950–51,Andy Phillip,G/F,Philadelphia Warriors,66,414,6.27,[5][10]


In [9]:
title = 'List of National Basketball Association annual steals leaders'
filename = 'steals.csv'
steals = get_table(title, filename, match='Larry Steele')
print(title)
steals.head()

List of National Basketball Association annual steals leaders


,Season,Player,Position[a],Team,Gamesplayed,Total steals,Steals per game,References
0,1973–74,Larry Steele,G/F,Portland Trail Blazers,81,217,2.68,[4][5]
1,1974–75,Rick Barry,F,Golden State Warriors,80,228,2.85,[6][7]
2,1975–76,Slick Watts,G,Seattle SuperSonics,82,261,3.18,[8][9]
3,1976–77,Don Buse,G,Indiana Pacers,81,281,3.47,[10][11]
4,1977–78,Ron Lee,G,Phoenix Suns,82,225,2.74,[12][13]


In [10]:
title = 'List of National Basketball Association annual blocks leaders'
filename = 'blocks.csv'
blocks = get_table(title, filename, match='Elmore Smith')
print(title)
blocks.head()

List of National Basketball Association annual blocks leaders


,Season,Player,Position[b],Team(s),Gamesplayed,Total blocks,Blocks per game,References
0,1973–74,Elmore Smith,C,Los Angeles Lakers,81,393,4.85,[4][5]
1,1974–75,Kareem Abdul-Jabbar,C,Milwaukee Bucks,65,212,3.26,[6][7]
2,1975–76,Kareem Abdul-Jabbar,C,Los Angeles Lakers,82,338,4.12,[7][8]
3,1976–77,Bill Walton,C/F,Portland Trail Blazers,65,211,3.25,[9][10]
4,1977–78,George T. Johnson,C/F,New Jersey Nets,81,274,3.38,[11][12]


In [11]:
title = 'Double (basketball)'
filename = 'triple-doubles.csv'
if not os.path.isfile(filename):
    shutil.copy(src='triple-doubles-table', dst=filename)
triple_doubles = get_table(title, filename, match='Oklahoma City Thunder')
print(title)
triple_doubles.head()

Double (basketball)


,Player,Triple-doubles,NBA season
0,Oscar Robertson,26,1960–61
1,Oscar Robertson,41,1961–62
2,Oscar Robertson,20,1962–63
3,Oscar Robertson,26,1963–64
4,Oscar Robertson,22,1964–65


In [12]:
title = 'List of National Basketball Association annual field goal percentage leaders'
filename = 'field_goal_pct.csv'
field_goal_pct = get_table(title, filename, match='Bob Feerick')
print(title)
field_goal_pct.head()

List of National Basketball Association annual field goal percentage leaders


,Season,Player,Position[a],Team,Gamesplayed,Field goals made,Field goals attempted,FG%,Ref.
0,1946–47,Bob Feerick,F/G,Washington Capitols,55,364,908,0.4009,[3]
1,1947–48,Buddy Jeannette,G,Baltimore Bullets,46,150,430,0.3488,[4]
2,1948–49,Arnie Risen,C/F,Rochester Royals,60,345,816,0.4228,[5]
3,1949–50,Alex Groza,C,Indianapolis Olympians,64,521,1090,0.4780,[6]
4,1950–51,Alex Groza,C,Indianapolis Olympians,66,492,1046,0.4704,[6]


In [13]:
title = 'List of National Basketball Association annual free throw percentage leaders'
filename = 'free_throw_pct.csv'
free_throw_pct = get_table(title, filename, match='Fred Scolari')
print(title)
free_throw_pct.head()

List of National Basketball Association annual free throw percentage leaders


,Season,Player,Position[a],Team,Gamesplayed,Free throws made,Free throws attempted,Free throw %,Ref.
0,1946–47,Fred Scolari,G,Washington Capitols,58,146,180,0.8111,[3]
1,1947–48,Bob Feerick,F/G,Washington Capitols,48,189,240,0.7875,[4]
2,1948–49,Bob Feerick,F/G,Washington Capitols,58,256,298,0.8591,[4]
3,1949–50,Max Zaslofsky,G/F,Chicago Stags,68,321,381,0.8425,[5]
4,1950–51,Joe Fulks,F/C,Philadelphia Warriors,66,378,442,0.8552,[6]


In [14]:
title = 'List of National Basketball Association annual three-point field goals leaders'
filename = 'three_point.csv'
three_point = get_table(title, filename, match='Brian Taylor')
print(title)
three_point.head()

List of National Basketball Association annual three-point field goals leaders


,Season,Player,Position[a],Team,Gamesplayed,3-pointfield goals made,3-pointfield goals attempted,3-pointfield goal %,Ref.
0,1979–80,Brian Taylor,G,San Diego Clippers,78,90,239,0.3766,[3]
1,1980–81,Mike Bratz,G,Cleveland Cavaliers,80,57,169,0.3373,[4]
2,1981–82,Don Buse,G,Indiana Pacers,82,78,193,0.4041,[5]
3,1982–83,Mike Dunleavy,G,San Antonio Spurs,79,67,194,0.3454,[6]
4,1983–84,Darrell Griffith,G,Utah Jazz,82,91,252,0.3611,[7]


In [15]:
title = 'List of National Basketball Association annual three-point field goal percentage leaders'
filename = 'three_point_pct.csv'
three_point_pct = get_table(title, filename, match='Fred Brown')
print(title)
three_point_pct.head()

List of National Basketball Association annual three-point field goal percentage leaders


,Season,Player,Position[a],Team,Gamesplayed,3-pointfield goals made,3-pointfield goals attempted,3P FG%,Ref.
0,1979–80,Fred Brown,G,Seattle SuperSonics,80,39,88,0.4432,[3]
1,1980–81,Brian Taylor,G,San Diego Clippers,80,44,115,0.3826,[4]
2,1981–82,Campy Russell,F,New York Knicks,77,25,57,0.4386,[5]
3,1982–83,Mike Dunleavy,G,Chicago Bulls,79,67,194,0.3454,[6]
4,1983–84,Darrell Griffith,G,Utah Jazz,82,91,252,0.3611,[7]


In [16]:
title = 'The NBA’s 50 Greatest Players list'
undefeated_50_greatest_list = []
with open('undefeated-50-greatest', 'r') as f:
    undefeated_50_greatest_list = f.read().splitlines()
undefeated_50_greatest = pd.DataFrame(undefeated_50_greatest_list, columns = ['Player'])
print(title)
undefeated_50_greatest.head()

The NBA’s 50 Greatest Players list


,Player
0,Kareem Abdul-Jabbar
1,Ray Allen
2,Paul Arizin
3,Charles Barkley
4,Rick Barry


In [17]:
def _get_championships(player_highlight):
    
    search_str = 'NBA champion'
    df = player_highlight[player_highlight.index.str.contains(search_str) == True]
    num = 0

    try:
        championships_str = df.index[0]

        m1 = re.match(r'.*As (a\s)?player:?\s+(\d+).+{}.*'.format(search_str), championships_str)
        m2 = re.match(r'.*As (a\s)?player:?.*{}.*'.format(search_str), championships_str)
        m3 = re.match(r'(\d+).+{}.*'.format(search_str), championships_str)
        m4 = re.match(r'{}.*'.format(search_str), championships_str)

        if m1:
            num = m1.group(2)
        if m2:
            if num == 0:
                num = 1
        if m3:
            if num == 0:
                num = m3.group(1) 
        if m4:
            if num == 0:
                num = 1
                
    except IndexError:
            num = 0

    return num

def get_player_info(player):

    player = force_to_unicode(player)
    print(player)

    while True:
        try:
            html = wp.page(player).html()
            break
        except wp.exceptions.DisambiguationError as e:
            print("Error: {0}".format(e))
        
        try:
            html = wp.page(player + ' (basketball)').html()
        except wp.exceptions.DisambiguationError as e:
            print("Error: {0}".format(e))
            
        break

    try:
        player_highlights = pd.read_html(html, match='Career information', attrs={'class':'infobox'}, index_col=0)
    except ValueError as e:
        print("Error: {0}".format(e))
        html = wp.page(player + ' (basketball)').html()
        player_highlights = pd.read_html(html, match='Career information', attrs={'class':'infobox'}, index_col=0)
        
    #only 1 of these will be returned, choose it
    player_highlight = player_highlights[0]

    #player position
    try:
        df = player_highlight.loc[['Position']]
        player_positions = df[1]
    except KeyError as e:
        print("Error: {0}".format(e))
        player_positions = []

    pos = u'Unknown'
    positions = ['CENTER', 'FORWARD', 'GUARD']
    for player_position in player_positions:
        if any (position in player_position.upper() for position in positions):
            pos = player_position

    #playing career
    try:
        df = player_highlight.loc[['Playing career']]
        career = df[1][0]
    except KeyError as e:
        print("Error: {0}".format(e))
        career = u'1900-2000'

    championships = int(_get_championships(player_highlight))

    return pos, career, championships

#position, career, championships = get_player_info('Kareem Abdul-Jabbar')
#position, career, championships = get_player_info('Dennis Rodman')
#position, career, championships = get_player_info('Earvin "Magic" Johnson')
#position, career, championships = get_player_info('Phil Jackson')
#position, career, championships = get_player_info('Tracy Murray')

#print(position, career, championships)

In [18]:
filename = 'players.csv'

if not os.path.isfile(filename):
    
    df = pd.DataFrame(columns=['Player'])

    df2 = championships[['Player']]
    df = pd.concat([df, df2], sort=False).drop_duplicates().reset_index(drop=True)

    df2 = scoring[['Player']]
    df = pd.concat([df, df2], sort=False).drop_duplicates().reset_index(drop=True)

    df2 = rebounding[['Player']]
    df = pd.concat([df, df2], sort=False).drop_duplicates().reset_index(drop=True)

    df2 = assists[['Player']]
    df = pd.concat([df, df2], sort=False).drop_duplicates().reset_index(drop=True)

    df2 = steals[['Player']]
    df = pd.concat([df, df2], sort=False).drop_duplicates().reset_index(drop=True)

    df2 = blocks[['Player']]
    df = pd.concat([df, df2], sort=False).drop_duplicates().reset_index(drop=True)

    df2 = triple_doubles[['Player']]
    df = pd.concat([df, df2], sort=False).drop_duplicates().reset_index(drop=True)

    df2 = field_goal_pct[['Player']]
    df = pd.concat([df, df2], sort=False).drop_duplicates().reset_index(drop=True)

    df2 = free_throw_pct[['Player']]
    df = pd.concat([df, df2], sort=False).drop_duplicates().reset_index(drop=True)

    df2 = three_point[['Player']]
    df = pd.concat([df, df2], sort=False).drop_duplicates().reset_index(drop=True)

    df2 = three_point_pct[['Player']]
    df = pd.concat([df, df2], sort=False).drop_duplicates().reset_index(drop=True)

    df2 = undefeated_50_greatest
    df = pd.concat([df, df2], sort=False).drop_duplicates().reset_index(drop=True)

    df['Position'], df['Career'], df['Championships'] = izip(*df['Player'].apply(get_player_info))

    df.to_csv('players.csv', header=True, index=False, encoding='utf-8')

# player position reassignments
player = pd.read_csv(filename)
player.Position[player.Player == 'Larry Bird'] = 'Small forward'
player.Position[player.Player == 'John Havlicek'] = 'Small forward'
player.Position[player.Player == 'Stephen Curry'] = 'Point guard / Shooting guard'
player.Position[player.Player == 'Bill Russell'] = 'Power forward / Center'
player.Position[player.Player == 'LeBron James'] = 'Small forward'

player.head()

,Player,Position,Career,Championships
0,Bill Russell,Power forward / Center,1956–1969,11
1,Sam Jones,Shooting guard,1957–1969,10
2,Tom Heinsohn,Power forward,1956–1965,8
3,K. C. Jones,Point guard,1958–1967,8
4,"Tom ""Satch"" Sanders",Power forward,1960–1973,8


In [19]:
def get_first_year(year):
    year = force_to_unicode(year)
    year = re.split('–', year)[0]
    return year


filename = 'seasons.csv'
title = 'List of National Basketball Association seasons'

if not os.path.isfile(filename):

    html = wp.page(title).html()
    df = pd.read_html(html, header=0, match='Washington Capitols')[0]
    df = df[df['Top seed'] != u'Playoffs']
    df = df[['Year[a]', 'No. of teams[d]']]
    
    df.rename(columns={'Year[a]': 'Year', 'No. of teams[d]': 'No. of teams'}, inplace=True)

    df['Year'] = df['Year'].apply(get_first_year)
    df.to_csv(filename, header=True, index=False, encoding='utf-8')

seasons = pd.read_csv(filename)
print(title)
seasons.head()

List of National Basketball Association seasons


,Year,No. of teams
0,Regular season,No. of teams[d]
1,1946,11
2,1947,8
3,1948,12
4,1949,17


In [20]:
def num_of_teams(row):
    career = row['Career']
    start = career[:4]
    end = career[-4:]
    if end == 'sent':
        now = datetime.now()
        end = now.year
    start = str(start)
    end = str(end)
    
    teams = seasons[(seasons.Year >= start) & (seasons.Year <= end)]['No. of teams']
    return teams.median()

player['No. of teams'] = player.apply(num_of_teams, axis=1)

In [21]:
def num_scoring_titles(row):
    num = scoring[scoring.Player == row.Player]
    return len(num)

player['Scoring'] = player.apply(num_scoring_titles, axis=1)

In [22]:
def num_rebounding_titles(row):
    num = rebounding[rebounding.Player == row.Player]
    return len(num)

player['Rebounding'] = player.apply(num_rebounding_titles, axis=1)

In [23]:
def num_assists_titles(row):
    num = assists[assists.Player == row.Player]
    return len(num)

player['Assists'] = player.apply(num_assists_titles, axis=1)

In [24]:
def num_steals_titles(row):
    num = steals[steals.Player == row.Player]
    return len(num)

player['Steals'] = player.apply(num_steals_titles, axis=1)

In [25]:
def num_blocks_titles(row):
    num = blocks[blocks.Player == row.Player]
    return len(num)

player['Blocks'] = player.apply(num_blocks_titles, axis=1)

In [26]:
def num_triple_doubles_titles(row):
    num = triple_doubles[triple_doubles.Player == row.Player]
    return len(num)

player['Triple doubles'] = player.apply(num_triple_doubles_titles, axis=1)

In [27]:
def num_field_goal_pct_titles(row):
    num = field_goal_pct[field_goal_pct.Player == row.Player]
    return len(num)

player['Field goal pct'] = player.apply(num_field_goal_pct_titles, axis=1)

In [28]:
def num_free_throw_pct_titles(row):
    num = free_throw_pct[free_throw_pct.Player == row.Player]
    return len(num)

player['Free throw pct'] = player.apply(num_free_throw_pct_titles, axis=1)

In [29]:
def num_three_point_titles(row):
    num = three_point[three_point.Player == row.Player]
    return len(num)

player['Three point'] = player.apply(num_three_point_titles, axis=1)

In [30]:
def num_three_point_pct_titles(row):
    num = three_point_pct[three_point_pct.Player == row.Player]
    return len(num)

player['Three point pct'] = player.apply(num_three_point_pct_titles, axis=1)

In [31]:
PRIMARY_SCALE_FACTOR = 1.0
SECONDARY_SCALE_FACTOR = 0.5
CHAMPIONSHIP_SCALE_FACTOR = 1.0

def get_primary_pts(row):
    pts = (row.Scoring + row.Rebounding
         + row.Assists + row.Steals 
         + row.Blocks)
    return pts

def get_secondary_pts(row):
    pts = (row['Triple doubles'] + row['Field goal pct']
         + row['Free throw pct'] + row['Three point']
         + row['Three point pct'])
    return pts

def get_championship_pts(row):
    pts = row.Championships
    return pts

def num_points(row):
    primary_pts = get_primary_pts(row)
    secondary_pts = get_secondary_pts(row)
    championship_pts = get_championship_pts(row)
    
    pts = primary_pts * PRIMARY_SCALE_FACTOR
    pts += secondary_pts * SECONDARY_SCALE_FACTOR
    if pts > 0:
        pts += min(championship_pts, pts) * CHAMPIONSHIP_SCALE_FACTOR
    return pts

player['Pts'] = player.apply(num_points, axis=1)

In [32]:
max_teams = player['No. of teams'].max()
def num_points_adj_teams(row):
    num = num_points(row)
    num *= row['No. of teams'] / max_teams
    return round(num, 2)

print('THE MAIN TABLE - Unsorted')
player['Pts adj teams'] = player.apply(num_points_adj_teams, axis=1)
player.head()

THE MAIN TABLE - Unsorted


,Player,Position,Career,Championships,No. of teams,Scoring,Rebounding,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
0,Bill Russell,Power forward / Center,1956–1969,11,9.0,0,4,0,0,0,1,0,0,0,0,9.0,2.7
1,Sam Jones,Shooting guard,1957–1969,10,9.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
2,Tom Heinsohn,Power forward,1956–1965,8,8.5,0,0,0,0,0,0,0,0,0,0,0.0,0.0
3,K. C. Jones,Point guard,1958–1967,8,9.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
4,"Tom ""Satch"" Sanders",Power forward,1960–1973,8,11.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0


In [33]:
print('The Greatest Players of ALL Time')
greatest = player.sort_values('Pts adj teams', ascending=False)
greatest.rename(columns={'Championships': 'Champ', 'Rebounding': 'Reb'}, inplace=True)
greatest = greatest.reset_index(drop=True)
greatest.index += 1
greatest

The Greatest Players of ALL Time


,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
1,Michael Jordan,Shooting guard,"1984–1993, 1995–1998, 2001–2003",6,27.0,10,0,0,3,0,1,0,0,0,0,19.5,17.55
2,"Earvin ""Magic"" Johnson",Point guard,"1979–1991, 1996",5,24.0,0,0,4,2,0,10,0,1,0,0,16.5,13.20
3,Dennis Rodman,Power forward,1986–2006,5,29.0,0,7,0,0,0,0,1,0,0,0,12.5,12.08
4,Jason Kidd,Point guard,1994–2013,1,29.5,0,0,5,0,0,11,0,0,0,0,11.5,11.31
5,Kareem Abdul-Jabbar,Center,1969–1989,6,22.0,2,1,0,0,4,2,1,0,0,0,14.5,10.63
6,Shaquille O'Neal,Center,1992–2011,4,29.0,2,0,0,0,0,0,10,0,0,0,11.0,10.63
7,Chris Paul,Point guard,2005–present,0,30.0,0,0,4,6,0,0,0,0,0,0,10.0,10.00
8,John Stockton,Point guard,1984–2003,0,27.0,0,0,9,2,0,0,0,0,0,0,11.0,9.90
9,Stephen Curry,Point guard / Shooting guard,2009–present,3,30.0,1,0,0,1,0,0,0,4,5,0,9.5,9.50
10,Wilt Chamberlain,Center,1959–1973,2,10.0,7,11,1,0,0,2,9,0,0,0,26.5,8.83


In [34]:
# Examine a player
greatest[greatest.Player == 'Tim Duncan']

,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
254,Tim Duncan,Power forward / Center,1997–2016,5,30.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0


In [35]:
# Examine a Category
greatest.sort_values('Field goal pct', ascending=False).head()

,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
6,Shaquille O'Neal,Center,1992–2011,4,29.0,2,0,0,0,0,0,10,0,0,0,11.0,10.63
10,Wilt Chamberlain,Center,1959–1973,2,10.0,7,11,1,0,0,2,9,0,0,0,26.5,8.83
24,DeAndre Jordan,Center,2008–present,0,30.0,0,2,0,0,0,0,5,0,0,0,4.5,4.50
73,Artis Gilmore,Center,1971–1989,0,23.0,0,0,0,0,0,0,4,0,0,0,2.0,1.53
32,Neil Johnston,Unknown,1900-2000,1,17.0,3,1,0,0,0,0,3,0,0,0,6.5,3.68


In [36]:
# Examine a Category, by Position
df = greatest[greatest.Position.str.contains('guard') == True]
df.sort_values('Scoring', ascending=False).head()

,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
1,Michael Jordan,Shooting guard,"1984–1993, 1995–1998, 2001–2003",6,27.0,10,0,0,3,0,1,0,0,0,0,19.5,17.55
37,George Gervin,Small forward / Shooting guard,1972–1990,0,23.0,4,0,0,0,0,0,0,0,0,0,4.0,3.07
13,Allen Iverson,Point guard / Shooting guard,1996–2011,0,29.5,4,0,0,3,0,0,0,0,0,0,7.0,6.88
28,James Harden,Shooting guard / Point guard,2009–present,0,30.0,2,0,1,0,0,0,0,0,2,0,4.0,4.00
62,Tracy McGrady,Shooting guard / Small forward,1997–2013,0,30.0,2,0,0,0,0,0,0,0,0,0,2.0,2.00


In [37]:
shooting_guard = greatest[(greatest.Position.str.contains('Shooting') == True) & (greatest.Scoring > 0)]
shooting_guard.head(10)

,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
1,Michael Jordan,Shooting guard,"1984–1993, 1995–1998, 2001–2003",6,27.0,10,0,0,3,0,1,0,0,0,0,19.5,17.55
9,Stephen Curry,Point guard / Shooting guard,2009–present,3,30.0,1,0,0,1,0,0,0,4,5,0,9.5,9.50
13,Allen Iverson,Point guard / Shooting guard,1996–2011,0,29.5,4,0,0,3,0,0,0,0,0,0,7.0,6.88
28,James Harden,Shooting guard / Point guard,2009–present,0,30.0,2,0,1,0,0,0,0,0,2,0,4.0,4.00
29,Kobe Bryant,Shooting guard,1996–2016,5,30.0,2,0,0,0,0,0,0,0,0,0,4.0,4.00
37,George Gervin,Small forward / Shooting guard,1972–1990,0,23.0,4,0,0,0,0,0,0,0,0,0,4.0,3.07
61,Dwyane Wade,Shooting guard,2003–2019,3,30.0,1,0,0,0,0,0,0,0,0,0,2.0,2.00
62,Tracy McGrady,Shooting guard / Small forward,1997–2013,0,30.0,2,0,0,0,0,0,0,0,0,0,2.0,2.00
84,Jerry West,Point guard / Shooting guard,1960–1974,1,12.0,1,0,1,0,0,0,0,0,0,0,3.0,1.20
127,Pete Maravich,Shooting guard,1970–1980,0,18.0,1,0,0,0,0,1,0,0,0,0,1.5,0.90


In [38]:
point_guard = greatest[(greatest.Position.str.contains('Point') == True ) & (greatest.Assists > 0)]
point_guard.head(10)

,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
2,"Earvin ""Magic"" Johnson",Point guard,"1979–1991, 1996",5,24.0,0,0,4,2,0,10,0,1,0,0,16.5,13.20
4,Jason Kidd,Point guard,1994–2013,1,29.5,0,0,5,0,0,11,0,0,0,0,11.5,11.31
7,Chris Paul,Point guard,2005–present,0,30.0,0,0,4,6,0,0,0,0,0,0,10.0,10.00
8,John Stockton,Point guard,1984–2003,0,27.0,0,0,9,2,0,0,0,0,0,0,11.0,9.90
11,Steve Nash,Point guard,1996–2015,2,30.0,0,0,5,0,0,0,0,2,0,0,8.0,8.00
14,Russell Westbrook,Point guard,2008–present,0,30.0,2,0,2,0,0,5,0,0,0,0,6.5,6.50
16,Rajon Rondo,Point guard,2006–present,1,30.0,0,0,3,1,0,2,0,0,0,0,6.0,6.00
25,Oscar Robertson,Point guard,1960–1974,1,12.0,0,0,6,0,0,6,0,2,0,0,11.0,4.40
27,Bob Cousy,Point guard,"1950–1963, 1969–1970",6,9.0,0,0,8,0,0,0,0,0,0,0,14.0,4.20
28,James Harden,Shooting guard / Point guard,2009–present,0,30.0,2,0,1,0,0,0,0,0,2,0,4.0,4.00


In [39]:
small_forward = greatest[(greatest.Position == 'Small forward')]
small_forward.head(5)

,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
15,Kevin Durant,Small forward,2007–present,2,30.0,4,0,0,0,0,0,0,1,0,0,6.5,6.50
21,LeBron James,Small forward,2003–present,3,30.0,1,0,0,0,0,3,0,0,0,0,5.0,5.00
22,Larry Bird,Small forward,1979–1992,3,23.0,0,0,0,0,0,1,0,4,2,0,6.5,4.98
33,Rick Barry,Small forward,1965–1980,1,17.0,1,0,0,1,0,0,0,6,0,0,6.0,3.40
46,Scottie Pippen,Small forward,1987–2008,6,29.0,0,0,0,1,0,1,0,0,0,0,3.0,2.90


In [40]:
power_forward = greatest[greatest.Position.str.contains('Power') == True]
power_forward.head(5)

,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
3,Dennis Rodman,Power forward,1986–2006,5,29.0,0,7,0,0,0,0,1,0,0,0,12.5,12.08
18,Kevin Garnett,Power forward / Center,1995–2016,1,30.0,0,4,0,0,0,1,0,0,0,0,5.5,5.50
30,Ben Wallace,Power forward / Center,1996–2012,1,30.0,0,2,0,0,1,0,0,0,0,0,4.0,4.00
31,Marcus Camby,Power forward / Center,1996–2013,0,30.0,0,0,0,0,4,0,0,0,0,0,4.0,4.00
39,Serge Ibaka,Power forward / Center,2007–present,1,30.0,0,0,0,0,2,0,0,0,0,0,3.0,3.00


In [41]:
center = greatest[greatest.Position == 'Center']
center.head(5)

,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
5,Kareem Abdul-Jabbar,Center,1969–1989,6,22.0,2,1,0,0,4,2,1,0,0,0,14.5,10.63
6,Shaquille O'Neal,Center,1992–2011,4,29.0,2,0,0,0,0,0,10,0,0,0,11.0,10.63
10,Wilt Chamberlain,Center,1959–1973,2,10.0,7,11,1,0,0,2,9,0,0,0,26.5,8.83
12,Dwight Howard,Center,2004–present,0,30.0,0,5,0,0,2,0,1,0,0,0,7.5,7.50
17,David Robinson,Center,1989–2003,2,29.0,1,1,0,0,1,2,0,0,0,0,6.0,5.80


In [42]:
df1 = shooting_guard[shooting_guard.Position == 'Shooting guard'][:1]
df2 = point_guard[point_guard.Position == 'Point guard'][:1]
df3 = small_forward.iloc[:1]
df4 = power_forward.iloc[:1]
df5 = center.iloc[:1]

first_team = pd.concat([df1, df2, df3, df4, df5], sort=False)
first_team

,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
1,Michael Jordan,Shooting guard,"1984–1993, 1995–1998, 2001–2003",6,27.0,10,0,0,3,0,1,0,0,0,0,19.5,17.55
2,"Earvin ""Magic"" Johnson",Point guard,"1979–1991, 1996",5,24.0,0,0,4,2,0,10,0,1,0,0,16.5,13.20
15,Kevin Durant,Small forward,2007–present,2,30.0,4,0,0,0,0,0,0,1,0,0,6.5,6.50
3,Dennis Rodman,Power forward,1986–2006,5,29.0,0,7,0,0,0,0,1,0,0,0,12.5,12.08
5,Kareem Abdul-Jabbar,Center,1969–1989,6,22.0,2,1,0,0,4,2,1,0,0,0,14.5,10.63


In [43]:
df1 = shooting_guard[shooting_guard.Position == 'Shooting guard'][1:2]
df2 = point_guard[point_guard.Position == 'Point guard'][1:2]
df3 = small_forward.iloc[1:2]
df4 = power_forward.iloc[1:2]
df5 = center.iloc[1:2]

second_team = pd.concat([df1, df2, df3, df4, df5], sort=False)
second_team

,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
29,Kobe Bryant,Shooting guard,1996–2016,5,30.0,2,0,0,0,0,0,0,0,0,0,4.0,4.00
4,Jason Kidd,Point guard,1994–2013,1,29.5,0,0,5,0,0,11,0,0,0,0,11.5,11.31
21,LeBron James,Small forward,2003–present,3,30.0,1,0,0,0,0,3,0,0,0,0,5.0,5.00
18,Kevin Garnett,Power forward / Center,1995–2016,1,30.0,0,4,0,0,0,1,0,0,0,0,5.5,5.50
6,Shaquille O'Neal,Center,1992–2011,4,29.0,2,0,0,0,0,0,10,0,0,0,11.0,10.63


In [44]:
present = greatest[greatest.Career.str.contains('present') == True]
present.head(10)

,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
7,Chris Paul,Point guard,2005–present,0,30.0,0,0,4,6,0,0,0,0,0,0,10.0,10.0
9,Stephen Curry,Point guard / Shooting guard,2009–present,3,30.0,1,0,0,1,0,0,0,4,5,0,9.5,9.5
12,Dwight Howard,Center,2004–present,0,30.0,0,5,0,0,2,0,1,0,0,0,7.5,7.5
14,Russell Westbrook,Point guard,2008–present,0,30.0,2,0,2,0,0,5,0,0,0,0,6.5,6.5
15,Kevin Durant,Small forward,2007–present,2,30.0,4,0,0,0,0,0,0,1,0,0,6.5,6.5
16,Rajon Rondo,Point guard,2006–present,1,30.0,0,0,3,1,0,2,0,0,0,0,6.0,6.0
21,LeBron James,Small forward,2003–present,3,30.0,1,0,0,0,0,3,0,0,0,0,5.0,5.0
24,DeAndre Jordan,Center,2008–present,0,30.0,0,2,0,0,0,0,5,0,0,0,4.5,4.5
28,James Harden,Shooting guard / Point guard,2009–present,0,30.0,2,0,1,0,0,0,0,0,2,0,4.0,4.0
38,Andre Drummond,Center,2012–present,0,30.0,0,3,0,0,0,0,0,0,0,0,3.0,3.0


In [45]:
df1 = present[present.Position.str.contains('Shooting') == True][:1] #SC at Shooting guard
df2 = present[present.Position.str.contains('Point') == True][:1]
df3 = present[present.Position.str.contains('Small') == True][:1]
df4 = present[present.Position.str.contains('Power') == True][:1]
df5 = present[present.Position.str.contains('Center') == True][:1]
present_team = pd.concat([df1, df2, df3, df4, df5], sort=False)
present_team

,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
9,Stephen Curry,Point guard / Shooting guard,2009–present,3,30.0,1,0,0,1,0,0,0,4,5,0,9.5,9.5
7,Chris Paul,Point guard,2005–present,0,30.0,0,0,4,6,0,0,0,0,0,0,10.0,10.0
15,Kevin Durant,Small forward,2007–present,2,30.0,4,0,0,0,0,0,0,1,0,0,6.5,6.5
39,Serge Ibaka,Power forward / Center,2007–present,1,30.0,0,0,0,0,2,0,0,0,0,0,3.0,3.0
12,Dwight Howard,Center,2004–present,0,30.0,0,5,0,0,2,0,1,0,0,0,7.5,7.5


In [46]:
print('Most Dominant Players of ALL Time')
dominant = greatest.sort_values('Pts', ascending=False)
dominant = dominant.reset_index(drop=True)
dominant.index += 1
dominant.head(5)

Most Dominant Players of ALL Time


,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
1,Wilt Chamberlain,Center,1959–1973,2,10.0,7,11,1,0,0,2,9,0,0,0,26.5,8.83
2,Michael Jordan,Shooting guard,"1984–1993, 1995–1998, 2001–2003",6,27.0,10,0,0,3,0,1,0,0,0,0,19.5,17.55
3,"Earvin ""Magic"" Johnson",Point guard,"1979–1991, 1996",5,24.0,0,0,4,2,0,10,0,1,0,0,16.5,13.20
4,Kareem Abdul-Jabbar,Center,1969–1989,6,22.0,2,1,0,0,4,2,1,0,0,0,14.5,10.63
5,Bob Cousy,Point guard,"1950–1963, 1969–1970",6,9.0,0,0,8,0,0,0,0,0,0,0,14.0,4.20


In [47]:
top_50 = greatest[:50].set_index(greatest.index[:50])
common = pd.merge(top_50, undefeated_50_greatest, how='inner', on=['Player'], right_index=True)
print('Common')
common

Common


,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
1,Michael Jordan,Shooting guard,"1984–1993, 1995–1998, 2001–2003",6,27.0,10,0,0,3,0,1,0,0,0,0,19.5,17.55
2,"Earvin ""Magic"" Johnson",Point guard,"1979–1991, 1996",5,24.0,0,0,4,2,0,10,0,1,0,0,16.5,13.20
4,Jason Kidd,Point guard,1994–2013,1,29.5,0,0,5,0,0,11,0,0,0,0,11.5,11.31
5,Kareem Abdul-Jabbar,Center,1969–1989,6,22.0,2,1,0,0,4,2,1,0,0,0,14.5,10.63
6,Shaquille O'Neal,Center,1992–2011,4,29.0,2,0,0,0,0,0,10,0,0,0,11.0,10.63
8,John Stockton,Point guard,1984–2003,0,27.0,0,0,9,2,0,0,0,0,0,0,11.0,9.90
9,Stephen Curry,Point guard / Shooting guard,2009–present,3,30.0,1,0,0,1,0,0,0,4,5,0,9.5,9.50
10,Wilt Chamberlain,Center,1959–1973,2,10.0,7,11,1,0,0,2,9,0,0,0,26.5,8.83
11,Steve Nash,Point guard,1996–2015,2,30.0,0,0,5,0,0,0,0,2,0,0,8.0,8.00
13,Allen Iverson,Point guard / Shooting guard,1996–2011,0,29.5,4,0,0,3,0,0,0,0,0,0,7.0,6.88


In [48]:
overrated = undefeated_50_greatest[~undefeated_50_greatest.Player.isin(common.Player)]
overrated = pd.merge(greatest, overrated, how='inner', on=['Player'], right_index=True)
print('Most Overrated')
overrated

Most Overrated


,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
61,Dwyane Wade,Shooting guard,2003–2019,3,30.0,1,0,0,0,0,0,0,0,0,0,2.0,2.00
70,Isiah Thomas,Point guard,1981–1994,2,24.0,0,0,1,0,0,0,0,0,0,0,2.0,1.60
72,Kevin McHale,Power forward,1980–1993,3,23.0,0,0,0,0,0,0,2,0,0,0,2.0,1.53
81,Charles Barkley,Power forward,1984–2000,0,27.0,0,1,0,0,0,1,0,0,0,0,1.5,1.35
84,Jerry West,Point guard / Shooting guard,1960–1974,1,12.0,1,0,1,0,0,0,0,0,0,0,3.0,1.20
94,Bob Pettit,Power forward / Center,1954–1965,1,8.5,2,1,0,0,0,0,0,0,0,0,4.0,1.13
96,Paul Arizin,Small forward,1950–1965,1,9.0,2,0,0,0,0,0,1,0,0,0,3.5,1.05
126,Jerry Lucas,Power forward / Center,1962–1974,1,14.0,0,0,0,0,0,1,1,0,0,0,2.0,0.93
235,Elgin Baylor,Small forward,1958–1971,0,9.0,0,0,0,0,0,1,0,0,0,0,0.5,0.15
237,Patrick Ewing,Center,1985–2002,0,27.0,0,0,0,0,0,0,0,0,0,0,0.0,0.00


In [49]:
underrated = top_50[~top_50.Player.isin(common.Player)]
underrated = pd.merge(greatest, underrated, how='inner', on=list(greatest), right_index=True)
print('Most Underrated')
underrated

Most Underrated


,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
3,Dennis Rodman,Power forward,1986–2006,5,29.0,0,7,0,0,0,0,1,0,0,0,12.5,12.08
7,Chris Paul,Point guard,2005–present,0,30.0,0,0,4,6,0,0,0,0,0,0,10.0,10.00
12,Dwight Howard,Center,2004–present,0,30.0,0,5,0,0,2,0,1,0,0,0,7.5,7.50
14,Russell Westbrook,Point guard,2008–present,0,30.0,2,0,2,0,0,5,0,0,0,0,6.5,6.50
16,Rajon Rondo,Point guard,2006–present,1,30.0,0,0,3,1,0,2,0,0,0,0,6.0,6.00
23,Dikembe Mutombo,Center,1991–2009,0,29.0,0,2,0,0,3,0,0,0,0,0,5.0,4.83
24,DeAndre Jordan,Center,2008–present,0,30.0,0,2,0,0,0,0,5,0,0,0,4.5,4.50
26,Bob McAdoo,Forward / Center,1972–1992,2,23.0,3,0,0,0,0,0,1,0,0,0,5.5,4.22
28,James Harden,Shooting guard / Point guard,2009–present,0,30.0,2,0,1,0,0,0,0,0,2,0,4.0,4.00
30,Ben Wallace,Power forward / Center,1996–2012,1,30.0,0,2,0,0,1,0,0,0,0,0,4.0,4.00


In [50]:
def is_old_timer(row):
    career = row['Career']
    start = career[:4]
    end = career[-4:]
    if end == 'sent':
        now = datetime.now()
        end = now.year
    start = int(start)
    end = int(end)
    return end < 1981

oldtimers = greatest[greatest.apply(is_old_timer, axis=1)]
oldtimers.head(10)

,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
10,Wilt Chamberlain,Center,1959–1973,2,10.0,7,11,1,0,0,2,9,0,0,0,26.5,8.83
25,Oscar Robertson,Point guard,1960–1974,1,12.0,0,0,6,0,0,6,0,2,0,0,11.0,4.40
27,Bob Cousy,Point guard,"1950–1963, 1969–1970",6,9.0,0,0,8,0,0,0,0,0,0,0,14.0,4.20
33,Rick Barry,Small forward,1965–1980,1,17.0,1,0,0,1,0,0,0,6,0,0,6.0,3.40
49,Bill Russell,Power forward / Center,1956–1969,11,9.0,0,4,0,0,0,1,0,0,0,0,9.0,2.70
50,George Mikan,Center,1946–1956,5,10.0,3,1,0,0,0,0,0,0,0,0,8.0,2.67
64,Bill Sharman,Shooting guard,1950–1961,4,8.5,0,0,0,0,0,0,0,7,0,0,7.0,1.98
82,Larry Steele,Guard,1971–1980,1,20.0,0,0,0,1,0,0,0,0,0,0,2.0,1.33
83,Ernie DiGregorio,Point guard,1973–1978,0,20.0,0,0,1,0,0,0,0,2,0,0,2.0,1.33
84,Jerry West,Point guard / Shooting guard,1960–1974,1,12.0,1,0,1,0,0,0,0,0,0,0,3.0,1.20


In [51]:
df1 = oldtimers[(oldtimers.Position.str.contains('Shooting') == True) & (oldtimers.Scoring > 0)][:1]
df2 = oldtimers[(oldtimers.Position.str.contains('Point') == True) & (oldtimers.Assists > 0)][:1]
df3 = oldtimers[oldtimers.Position.str.contains('Small') == True][:1]
df4 = oldtimers[oldtimers.Position.str.contains('Power') == True][:1]
df5 = oldtimers[oldtimers.Position.str.contains('Center') == True][:1]
oldtimers_team = pd.concat([df1, df2, df3, df4, df5], sort=False)
oldtimers_team

,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams
84,Jerry West,Point guard / Shooting guard,1960–1974,1,12.0,1,0,1,0,0,0,0,0,0,0,3.0,1.20
25,Oscar Robertson,Point guard,1960–1974,1,12.0,0,0,6,0,0,6,0,2,0,0,11.0,4.40
33,Rick Barry,Small forward,1965–1980,1,17.0,1,0,0,1,0,0,0,6,0,0,6.0,3.40
49,Bill Russell,Power forward / Center,1956–1969,11,9.0,0,4,0,0,0,1,0,0,0,0,9.0,2.70
10,Wilt Chamberlain,Center,1959–1973,2,10.0,7,11,1,0,0,2,9,0,0,0,26.5,8.83


NOTE: The code below normalizes for player position.  I don't like the way that turned out.  
I'm leaving it in only because the code is useful.

In [52]:
def get_scale_factor():
    pts_by_pos = {'Shooting': 0, 'Point': 0, 'Small': 0, 'Power': 0, 'Center': 0}

    # normalize by position
    def get_pts_by_pos(position):
        df = greatest[greatest.Position.str.contains(position) == True][:10]
        pts = df['Pts adj teams'].median()
        return pts

    for key, _ in pts_by_pos.items():
        pts_by_pos[key] = get_pts_by_pos(key)

    max_pts_by_pos = max(pts_by_pos.values())

    sf = pts_by_pos.copy()
    for key in sf:    
        sf[key] = max_pts_by_pos / sf[key]
    return sf

In [53]:
sf = get_scale_factor()

def num_points_adj_position(row):
    for key, value in sf.items():
        if key in row.Position:
            pts = row['Pts adj teams']
            pts *= value
            return pts.round(2)

print('The Greatest Players of ALL Time - normalized by position')
norm_greatest = greatest.copy()
norm_greatest['Pts adj pos'] = norm_greatest.apply(num_points_adj_position, axis=1)
norm_greatest = norm_greatest.sort_values('Pts adj pos', ascending=False)
norm_greatest = norm_greatest.reset_index(drop=True)
norm_greatest.index += 1
norm_greatest.head(50)

The Greatest Players of ALL Time - normalized by position


,Player,Position,Career,Champ,No. of teams,Scoring,Reb,Assists,Steals,Blocks,Triple doubles,Field goal pct,Free throw pct,Three point,Three point pct,Pts,Pts adj teams,Pts adj pos
1,Michael Jordan,Shooting guard,"1984–1993, 1995–1998, 2001–2003",6,27.0,10,0,0,3,0,1,0,0,0,0,19.5,17.55,41.62
2,Dennis Rodman,Power forward,1986–2006,5,29.0,0,7,0,0,0,0,1,0,0,0,12.5,12.08,35.23
3,Stephen Curry,Point guard / Shooting guard,2009–present,3,30.0,1,0,0,1,0,0,0,4,5,0,9.5,9.50,22.53
4,Kevin Durant,Small forward,2007–present,2,30.0,4,0,0,0,0,0,0,1,0,0,6.5,6.50,18.74
5,Kareem Abdul-Jabbar,Center,1969–1989,6,22.0,2,1,0,0,4,2,1,0,0,0,14.5,10.63,16.46
6,Shaquille O'Neal,Center,1992–2011,4,29.0,2,0,0,0,0,0,10,0,0,0,11.0,10.63,16.46
7,Allen Iverson,Point guard / Shooting guard,1996–2011,0,29.5,4,0,0,3,0,0,0,0,0,0,7.0,6.88,16.31
8,Kevin Garnett,Power forward / Center,1995–2016,1,30.0,0,4,0,0,0,1,0,0,0,0,5.5,5.50,16.04
9,LeBron James,Small forward,2003–present,3,30.0,1,0,0,0,0,3,0,0,0,0,5.0,5.00,14.42
10,Larry Bird,Small forward,1979–1992,3,23.0,0,0,0,0,0,1,0,4,2,0,6.5,4.98,14.36
